# ConvLSTM model

Import libraries and modules.

In [1]:
import torch
import os
# import imageio

import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
# import torch.nn.functional as F

# from numba import jit, prange
# from PIL import Image
# from sklearn.preprocessing import MinMaxScaler
# from torch.utils.data.dataset import random_split
from torchsummary import summary
from torch.utils.data import DataLoader
# from matplotlib.colors import TwoSlopeNorm

from load_datasets import *
from ConvLSTM_pytorch import *
from encode_decode_csv import decode_from_csv

Check if GPU is available.

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


The following lines create variables to more easily specify what we use the model for (i.e., train and validate, test with dataset 1 and so on) in the following functions.

In [3]:
train_val = 'train_val'
test1 = 'test1'
test2 = 'test2'
test3 = 'test3'

Load data from decoder.

!!! inputs and targets in train_val are stored not in the original manner. index 0 = 1, index 1 = 10, index 11 = 2, index 12 = 20 etc.

In [4]:
inputs, targets = decode_from_csv(train_val)

Restored inputs Shape: torch.Size([80, 3, 64, 64])
Restored targets Shape: torch.Size([80, 97, 2, 64, 64])


In [5]:
inps1, targs1 = decode_from_csv(test1)

Restored inputs Shape: torch.Size([20, 3, 64, 64])
Restored targets Shape: torch.Size([20, 97, 2, 64, 64])


In [6]:
inps2, targs2 = decode_from_csv(test2)

Restored inputs Shape: torch.Size([21, 3, 64, 64])
Restored targets Shape: torch.Size([21, 97, 2, 64, 64])


In [7]:
inps3, targs3 = decode_from_csv(test3)

Restored inputs Shape: torch.Size([10, 3, 128, 128])
Restored targets Shape: torch.Size([10, 97, 2, 128, 128])
